## 🚀 Scale Your MVP

### 1. Full Coverage: The Magnificent Seven
Target companies:

- Apple, Microsoft, Amazon, Google (Alphabet), Meta, Nvidia, Tesla
- Benchmark: S&P 500
### 2. Database PostgreSQL
Create a relational database with the following tables:

- **stock_prices:** historical prices per company

- **market_events:** price events (+/-5%) or key news

- **news_raw:** all raw news articles

- **news_clean:** cleaned and filtered relevant news

- **gpt_analysis:** GPT sentiment and investment recommendation output

- **backtest_results:** ROI of simulated investments after event

### 3. Visualizations
Using Plotly, Seaborn, or Power BI:

📈 Price chart with event markers and GPT sentiment

📊 ROI comparison: real investment vs. investment after news

📊 Event impact ranking by company

🧠 Correlation between GPT relevance and actual ROI

### 4. Pipeline Automation
With argparse or modular scripts that do:

- Event detection

- News retrieval

- News filtering + cleaning

- Prompt generation

- GPT evaluation

- Result storage

- ROI simulation

### 5. Comparison Against S&P 500
Calculate the ROI of the S&P 500 on the same event dates

Compare each company's ROI with the index

Did the event impact only the company or also the market?

### 6. Heuristic-Based Decision Making
Define a simple decision tree based on GPT output:

- if relevance > 0.7 and sentiment == "Bullish" and PER < 20:
    return "Buy"
- elif relevance > 0.7 and sentiment == "Bullish" and PER > 35:
    return "Risk"

In [2]:
symbols = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]

In [1]:
import os
import openai
import pandas as pd
from alpaca_trade_api.rest import REST
import pandas as pd
from datetime import datetime, timedelta
import requests
from dotenv import load_dotenv
import os

load_dotenv()

ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_API_SECRET = os.getenv("ALPACA_API_SECRET")
ALPACA_BASE_URL = os.getenv("ALPACA_BASE_URL")
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY

### 1. EVENT IDENTIFIER - AAPL
Down/upsides  of over 5% in a Day

In [6]:
# === CONEXIÓN ===
api = REST(ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_BASE_URL, api_version="v2")

def get_price_data(symbol: str, start_date: str, end_date: str, pct_change):
    df = api.get_bars(symbol, timeframe='1D', start=start_date, end=end_date).df
    # df = df[df['symbol'] == symbol]  ❌ innecesaria
    df['pct_change'] = df['close'].pct_change() * 100
    df['event'] = df['pct_change'].abs() > pct_change
    return df


def detect_price_events(df):
    return df[df['event']].copy()

In [9]:
if __name__ == "__main__":
    symbol = "SPY"
    df = get_price_data(symbol, "2000-01-01", "2025-03-31", 5)
    events = detect_price_events(df)
    print(events[['close', 'pct_change']])

                             close  pct_change
timestamp                                     
2020-03-09 04:00:00+00:00  276.320   -7.097468
2020-03-12 04:00:00+00:00  255.240   -6.931632
2020-03-13 04:00:00+00:00  270.200    5.861150
2020-03-16 04:00:00+00:00  241.065  -10.782754
2020-03-17 04:00:00+00:00  254.190    5.444590
2020-03-18 04:00:00+00:00  235.690   -7.278020
2020-03-24 04:00:00+00:00  242.000    8.676127
2020-03-26 04:00:00+00:00  260.930    5.729568
2020-04-06 04:00:00+00:00  264.740    6.771527
2020-06-11 04:00:00+00:00  300.690   -5.748676
2022-11-10 05:00:00+00:00  394.690    5.495416


In [10]:
if __name__ == "__main__":
    symbol = "AAPL"
    df = get_price_data(symbol, "2022-01-01", "2025-03-31", 5)
    events = detect_price_events(df)
    print(events[['close', 'pct_change']])

                            close  pct_change
timestamp                                    
2022-01-28 05:00:00+00:00  170.33    6.977767
2022-05-05 04:00:00+00:00  156.77   -5.571618
2022-05-11 04:00:00+00:00  146.50   -5.184130
2022-05-18 04:00:00+00:00  140.82   -5.641919
2022-09-13 04:00:00+00:00  153.84   -5.867956
2022-10-28 04:00:00+00:00  155.74    7.555249
2022-11-10 05:00:00+00:00  146.87    8.897457
2024-05-03 04:00:00+00:00  183.38    5.981622
2024-06-11 04:00:00+00:00  207.15    7.264913


### 2. GET NEWS for EVENT - AAPL +7.27%  in 11-06-2024    
Check out the News from AlphaVantage that came out near to this EVENT

In [52]:
from datetime import datetime, timedelta
import pandas as pd
import requests
import os

def get_news_sentiments_for_event(symbol, event_date, api_key):
    dt = datetime.strptime(event_date, "%Y-%m-%d")
    start = (dt - timedelta(days=7)).strftime("%Y%m%dT0000")
    end = dt.strftime("%Y%m%dT2359")

    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": symbol,
        "time_from": start,
        "time_to": end,
        "limit": 50,
        "apikey": api_key
    }

    print(f"🔍 Buscando noticias desde {start} hasta {end} para {symbol}...")
    res = requests.get(url, params=params)
    data = res.json()
    
    if "feed" in data:
        df = pd.DataFrame(data["feed"])
        return df
    else:
        print("❌ No se encontraron noticias o alcanzaste el límite de la API.")
        return pd.DataFrame()

# Parámetros de prueba
SYMBOL = "AAPL"
EVENT_DATE = "2024-06-11"
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")  # O pon tu clave directamente aquí como string

# Ejecutar prueba
news_df = get_news_sentiments_for_event(SYMBOL, EVENT_DATE, API_KEY)

if not news_df.empty:
    print(f"✅ {len(news_df)} noticias encontradas.")
    print(news_df[['title', 'summary']].head(5))
else:
    print("⚠️ No se encontraron noticias.")


🔍 Buscando noticias desde 20240604T0000 hasta 20240611T2359 para AAPL...
✅ 50 noticias encontradas.
                                               title  \
0  Time to Buy Apple's  ( AAPL )  Stock for Highe...   
1  S&P, Nasdaq Notch New Closing Highs Ahead of C...   
2  Event Contract Trading Signals Uncertainty In ...   
3         Best ETF Ideas for the Second Half of 2024   
4  Apple Soars, Drives Nasdaq To High; Fed, Infla...   

                                             summary  
0  Apple's (AAPL) stock spiked +7% today to fresh...  
1  While Apple (AAPL) helped the Nasdaq and S&P h...  
2  The chance of courts seeing Apple Inc AAPL as ...  
3  We discuss the market outlook and investing st...  
4  Dow Jones Futures: Apple Drives S&P 500 To Hig...  


In [53]:
news_df
news_df.to_csv(f'news_{SYMBOL}_{EVENT_DATE}.csv', sep=';', index=False)

### 3. FILTER NEWS for EVENT - AAPL +7.27%  in 11-06-2024    
Filter out the News from AlphaVantage that have a relevance score of over 0.5

In [54]:
import pandas as pd
import ast
from datetime import datetime

# === CONFIGURACIÓN ===
CSV_INPUT = "news_AAPL_2024-06-11.csv"  # Cambiar si se analiza otro archivo
CSV_OUTPUT = "news_AAPL_2024-06-11_clean.csv"
SEPARADOR = ";"
TICKER_OBJETIVO = "AAPL"
RELEVANCIA_MINIMA = 0.5

# === FUNCIONES ===
def extract_aapl_relevance(ticker_sentiment, ticker="AAPL"):
    try:
        sentiments = ast.literal_eval(ticker_sentiment)
        for entry in sentiments:
            if entry['ticker'] == ticker:
                return float(entry['relevance_score'])
    except (ValueError, SyntaxError, KeyError, TypeError):
        return None

# === PROCESO PRINCIPAL ===
def limpiar_noticias_alpha(csv_input, csv_output, ticker_objetivo, min_relevancia):
    df = pd.read_csv(csv_input, sep=SEPARADOR)
    df['relevancia_AAPL'] = df['ticker_sentiment'].apply(lambda x: extract_aapl_relevance(x, ticker_objetivo))

    # Filtrar por relevancia
    df_filtrado = df[df['relevancia_AAPL'] > min_relevancia]

    # Seleccionar columnas útiles y renombrar
    df_limpio = df_filtrado[[
        'title', 'summary', 'url', 'time_published',
        'overall_sentiment_label', 'overall_sentiment_score', 'relevancia_AAPL'
    ]].copy()

    df_limpio['time_published'] = pd.to_datetime(df_limpio['time_published'], format='%Y%m%dT%H%M%S')

    df_limpio.rename(columns={
        'title': 'titulo',
        'summary': 'resumen',
        'url': 'url',
        'time_published': 'fecha',
        'overall_sentiment_label': 'sentimiento',
        'overall_sentiment_score': 'score_sentimiento'
    }, inplace=True)

    # Guardar CSV limpio
    df_limpio.to_csv(csv_output, sep=SEPARADOR, index=False)
    print(f"✅ Noticias limpiadas guardadas en: {csv_output}")

# === EJECUCIÓN ===
if __name__ == "__main__":
    limpiar_noticias_alpha(CSV_INPUT, CSV_OUTPUT, TICKER_OBJETIVO, RELEVANCIA_MINIMA)


✅ Noticias limpiadas guardadas en: news_AAPL_2024-06-11_clean.csv


### 4. GET OpenAI Interpreation for EVENT - AAPL +7.27%  in 11-06-2024    
Send him the news and evaluate them

In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key="")

print("🔑 Longitud de clave:", len(client.api_key))

# Cargar archivo limpio
df = pd.read_csv("news_AAPL_2024-06-11_clean.csv", sep=";")
noticia = df.iloc[0]

# Datos del evento
fecha_evento = "2024-06-11"
precio = 207.15
variacion = 7.26
ticker = "AAPL"
sector = "Tecnología"
indice_mercado = "S&P 500"
contexto_macro = """
En junio de 2024, el contexto económico está marcado por recuperación tras un periodo inflacionario,
con políticas monetarias moderadas y una expectativa de bajadas de tipos en EE.UU. La tecnología lidera las subidas del mercado.
"""

prompt = f"""
Actúa como un analista financiero experto. A continuación tienes una noticia del {fecha_evento} sobre la empresa {ticker}, que pertenece al sector {sector}.
Ese día la acción tuvo una subida de +{variacion:.2f}% y cerró en {precio} USD.

---

📌 Noticia:
Título: {noticia['titulo']}
Resumen: {noticia['resumen']}
Fuente: {noticia['url']}

---

📊 Contexto macroeconómico:
{contexto_macro}

📈 El {indice_mercado} también subió ligeramente ese día.

---

🔍 Por favor responde:
1. ¿Cuál es el sentimiento principal de esta noticia? Usa una sola palabra: Bullish / Neutral / Bearish / Cautious.
2. ¿Cuál sería un score de sentimiento del 0 al 1?
3. ¿Crees que esta noticia influyó significativamente en la subida del precio de la acción ese día? Estima un valor entre 0 (no influyó) y 1 (fue completamente responsable).
4. Justifica brevemente tu razonamiento considerando el contenido de la noticia, el contexto macro y el comportamiento general del mercado.

Responde en formato claro y estructurado.
"""

response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

print(response.choices[0].message.content)


🔑 Longitud de clave: 164
1. Sentimiento principal: Bullish

2. Score de sentimiento: 0.8

3. Influencia en la subida del precio: 0.7

4. Justificación:
La noticia presenta un tono claramente optimista, destacando una asociación estratégica entre Apple y OpenAI para integrar ChatGPT en sus productos, lo cual sugiere innovación y potencial crecimiento. La subida del +7.26% en la acción refleja una fuerte reacción positiva del mercado, probablemente impulsada por esta noticia y la percepción de avances tecnológicos que podrían potenciar las ventas y la cuota de mercado de Apple. Además, en el contexto macroeconómico favorable, con recuperación económica, políticas monetarias moderadas y un sector tecnológico en auge, es probable que los inversores hayan interpretado esta noticia como una señal de crecimiento sostenido. Aunque otros factores como el comportamiento general del mercado y la tendencia alcista del S&P 500 también contribuyeron, la noticia en sí parece haber sido un catalizador

### 5. Trading/Holding Strategy for NEWS afer EVENT - AAPL +7.27%  in 11-06-2024    

In [57]:
from datetime import date, timedelta
from alpaca_trade_api.rest import REST
import os

# === CONFIGURACIÓN ===
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY") or "TU_API_KEY"
ALPACA_API_SECRET = os.getenv("ALPACA_API_SECRET") or "TU_SECRET"
ALPACA_BASE_URL = "https://paper-api.alpaca.markets"
SYMBOL = "AAPL"
buy_date = date(2024, 6, 4)
sell_date = date(2024, 6, 18)

# === CONEXIÓN API ===
api = REST(ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_BASE_URL, api_version='v2')

# === FUNCIONES ===
def get_close_price(symbol, target_date):
    barset = api.get_bars(symbol, timeframe="1Day", start=target_date, end=target_date + timedelta(days=1)).df
    if not barset.empty:
        return barset.iloc[0]['close']
    else:
        return None

# === OBTENER PRECIOS ===
price_buy = get_close_price(SYMBOL, buy_date)
price_sell = get_close_price(SYMBOL, sell_date)

if price_buy and price_sell:
    roi = (price_sell - price_buy) / price_buy
    print(f"💰 Compra el {buy_date}: ${price_buy:.2f}")
    print(f"💰 Venta el {sell_date}: ${price_sell:.2f}")
    print(f"📈 ROI: {roi*100:.2f}%")
else:
    print("❌ No se pudieron obtener ambos precios.")


💰 Compra el 2024-06-04: $194.35
💰 Venta el 2024-06-18: $214.29
📈 ROI: 10.26%


In [62]:
from datetime import datetime, timedelta
import pandas as pd
from alpaca_trade_api.rest import REST
import os

# === CONFIGURACIÓN ===
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY") or "TU_API_KEY"
ALPACA_API_SECRET = os.getenv("ALPACA_API_SECRET") or "TU_SECRET"
ALPACA_BASE_URL = "https://paper-api.alpaca.markets"

SYMBOL = "AAPL"
EVENT_DATE = "2024-06-11"  # fecha de análisis

# === CONEXIÓN ===
api = REST(ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_BASE_URL, api_version="v2")

# === FUNCIONES ===
def get_price_on(date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    start = date.strftime("%Y-%m-%d")
    end = (date + timedelta(days=1)).strftime("%Y-%m-%d")
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars.iloc[0]['close'] if not bars.empty else None


def get_ytd_mean_price(year):
    start = f"{year}-01-01"
    end = EVENT_DATE
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars['close'].mean() if not bars.empty else None

def get_min_12m_price():
    end_dt = datetime.strptime(EVENT_DATE, "%Y-%m-%d")
    start_dt = end_dt - timedelta(days=365)
    start = start_dt.strftime("%Y-%m-%d")
    end = end_dt.strftime("%Y-%m-%d")
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars['close'].min() if not bars.empty else None


def get_ma200():
    end_dt = datetime.strptime(EVENT_DATE, "%Y-%m-%d")
    start_dt = end_dt - timedelta(days=300)
    start = start_dt.strftime("%Y-%m-%d")
    end = end_dt.strftime("%Y-%m-%d")
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars['close'].rolling(window=200).mean().iloc[-1] if len(bars) >= 200 else None


# === EJECUCIÓN ===
price_event_day = get_price_on(EVENT_DATE)
ytd_avg = get_ytd_mean_price(EVENT_DATE[:4])
min_12m = get_min_12m_price()
ma_200 = get_ma200()

# === MOSTRAR RESULTADOS ===
print("📊 KPIs desde Alpaca para", SYMBOL, "en", EVENT_DATE)
print("Precio día evento:", price_event_day)
print("Media YTD:", ytd_avg)
print("Mínimo últimos 12M:", min_12m)
print("Media móvil 200 días:", ma_200)

# Opcional: exportar a dict o CSV para usar en prompt


📊 KPIs desde Alpaca para AAPL en 2024-06-11
Precio día evento: 207.15
Media YTD: 181.22437499999998
Mínimo últimos 12M: 165.0
Media móvil 200 días: 181.93985


In [64]:
from datetime import datetime, timedelta
import pandas as pd
from alpaca_trade_api.rest import REST
import os
import requests

# === CONFIGURACIÓN ===
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY") or "TU_API_KEY"
ALPACA_API_SECRET = os.getenv("ALPACA_API_SECRET") or "TU_SECRET"
ALPACA_BASE_URL = "https://paper-api.alpaca.markets"
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY") or "TU_ALPHA_KEY"

SYMBOL = "AAPL"
EVENT_DATE = "2024-06-04"

# === CONEXIÓN ===
api = REST(ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_BASE_URL, api_version="v2")

# === FUNCIONES ===
def get_price_on(date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    start = date.strftime("%Y-%m-%d")
    end = (date + timedelta(days=1)).strftime("%Y-%m-%d")
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars.iloc[0]['close'] if not bars.empty else None

def get_ytd_mean_price(year):
    start = f"{year}-01-01"
    end = EVENT_DATE
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars['close'].mean() if not bars.empty else None

def get_min_12m_price():
    end_dt = datetime.strptime(EVENT_DATE, "%Y-%m-%d")
    start_dt = end_dt - timedelta(days=365)
    start = start_dt.strftime("%Y-%m-%d")
    end = end_dt.strftime("%Y-%m-%d")
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars['close'].min() if not bars.empty else None

def get_ma200():
    end_dt = datetime.strptime(EVENT_DATE, "%Y-%m-%d")
    start_dt = end_dt - timedelta(days=300)
    start = start_dt.strftime("%Y-%m-%d")
    end = end_dt.strftime("%Y-%m-%d")
    bars = api.get_bars(SYMBOL, timeframe="1Day", start=start, end=end).df
    return bars['close'].rolling(window=200).mean().iloc[-1] if len(bars) >= 200 else None

def get_fundamentals(symbol):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url).json()
    return {
        "PER": response.get("PERatio"),
        "EPS": response.get("EPS"),
        "FCF": response.get("FreeCashFlow"),
        "NetIncome": response.get("NetIncomeTTM"),
        "ROE": response.get("ReturnOnEquityTTM"),
        "ROIC": response.get("ReturnOnAssetsTTM"),
        "DebtToEBITDA": response.get("EVToEBITDA"),
        "GrossMargin": response.get("GrossProfitTTM"),
        "OperatingMargin": response.get("OperatingMarginTTM"),
        "CAGR": response.get("RevenueTTM"),  # Placeholder si no hay CAGR
        "AnalystEstimates": response.get("AnalystTargetPrice"),
    }

# === EJECUCIÓN ===
price_event_day = get_price_on(EVENT_DATE)
ytd_avg = get_ytd_mean_price(EVENT_DATE[:4])
min_12m = get_min_12m_price()
ma_200 = get_ma200()
fundamentals = get_fundamentals(SYMBOL)

# === CREAR DATAFRAME ===
data = {
    "Fecha evento": EVENT_DATE,
    "Ticker": SYMBOL,
    "Precio evento": price_event_day,
    "Media YTD": ytd_avg,
    "Mínimo 12M": min_12m,
    "Media 200 días": ma_200,
    **fundamentals
}

df_kpis = pd.DataFrame([data])
print(df_kpis)

# Guardar si se desea
df_kpis.to_csv("kpis_AAPL_2024-06-04.csv", sep=";", index=False)


  Fecha evento Ticker  Precio evento   Media YTD  Mínimo 12M  Media 200 días  \
0   2024-06-04   AAPL         194.35  180.463738       165.0        181.4276   

     PER   EPS   FCF NetIncome    ROE   ROIC DebtToEBITDA   GrossMargin  \
0  31.76  6.29  None      None  1.365  0.225        21.44  184102994000   

  OperatingMargin          CAGR AnalystEstimates  
0           0.345  395760009000           237.87  


In [67]:
import pandas as pd

# === Cargar KPIs (reemplaza si ya tienes el DataFrame en memoria) ===
df = pd.read_csv("kpis_AAPL_2024-06-04.csv", sep=";")
row = df.iloc[0]

# === Parámetros noticia ===
titulo = "Apple anuncia integración con ChatGPT en todos sus dispositivos"
resumen = "Apple presentó una integración directa con ChatGPT como parte de su estrategia de inteligencia artificial durante la WWDC 2024, provocando un aumento del 7.2% en su cotización bursátil."
url = "https://www.apple.com/newsroom/2024/06/apple-integrates-chatgpt"

# === Plantilla de prompt ===
prompt_value = f"""
Actúa como un analista financiero experto. Quiero que analices si una inversión en la acción {row['Ticker']} era una buena decisión en la fecha {row['Fecha evento']}, combinando análisis fundamental (value investing) con el impacto de una noticia relevante.

---

📊 Datos fundamentales del momento:
- PER: {row['PER']}
- EPS: {row['EPS']}
- FCF: {row['FCF']}
- Net Income: {row['NetIncome']}
- ROE: {row['ROE']}
- ROIC: {row['ROIC']}
- Deuda/EBITDA: {row['DebtToEBITDA']}
- Margen Bruto: {row['GrossMargin']}
- Margen Operativo: {row['OperatingMargin']}
- CAGR de ingresos (estimado): {row['CAGR']}
- Estimación analistas: {row['AnalystEstimates']}
- Precio evento: {row['Precio evento']}
- Media YTD: {row['Media YTD']}
- Mínimo 12M: {row['Mínimo 12M']}
- Media móvil 200 días: {row['Media 200 días']}

---

📰 Noticia del día:
Título: {titulo}
Resumen: {resumen}
Fuente: {url}

---

🔍 Análisis requerido:

1. Interpreta cada métrica: ¿qué indica sobre la empresa?
2. ¿Qué sugiere la noticia sobre el futuro de la empresa?
3. ¿Están los fundamentos alineados con el optimismo/pesimismo de la noticia?
4. ¿La empresa estaba infravalorada, sobrevalorada o correctamente valorada?
5. ¿Era una buena decisión comprar ese día según el enfoque value?
6. ¿Qué indicador tuvo más peso en tu conclusión (PER bajo, FCF, ROIC, etc.)?
7. Concluye con una recomendación: Comprar / Mantener / Vender.

El análisis debe ser claro, justificado y estructurado para un inversor de largo plazo.
""".strip()

print(prompt_value)


Actúa como un analista financiero experto. Quiero que analices si una inversión en la acción AAPL era una buena decisión en la fecha 2024-06-04, combinando análisis fundamental (value investing) con el impacto de una noticia relevante.

---

📊 Datos fundamentales del momento:
- PER: 31.76
- EPS: 6.29
- FCF: nan
- Net Income: nan
- ROE: 1.365
- ROIC: 0.225
- Deuda/EBITDA: 21.44
- Margen Bruto: 184102994000
- Margen Operativo: 0.345
- CAGR de ingresos (estimado): 395760009000
- Estimación analistas: 237.87
- Precio evento: 194.35
- Media YTD: 180.46373831775696
- Mínimo 12M: 165.0
- Media móvil 200 días: 181.4276

---

📰 Noticia del día:
Título: Apple anuncia integración con ChatGPT en todos sus dispositivos
Resumen: Apple presentó una integración directa con ChatGPT como parte de su estrategia de inteligencia artificial durante la WWDC 2024, provocando un aumento del 7.2% en su cotización bursátil.
Fuente: https://www.apple.com/newsroom/2024/06/apple-integrates-chatgpt

---

🔍 Análisis 

In [68]:

response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role": "user", "content": prompt_value}],
    temperature=0.3
)

print(response.choices[0].message.content)


Análisis financiero y estratégico de Apple (AAPL) al 2024-06-04

---

**1. Interpretación de las métricas fundamentales**

- **PER (31.76):**  
  Un PER de 31.76 indica que los inversores están pagando aproximadamente 32 veces las ganancias por acción. Esto es relativamente alto en comparación con empresas tradicionales, pero en tecnología y especialmente en Apple, puede considerarse en línea con su perfil de crecimiento y valor de marca. Sin embargo, no es un PER bajo, lo que sugiere que la acción no está claramente infravalorada desde un enfoque de value investing.

- **EPS (6.29):**  
  La ganancia por acción es sólida, pero sin un contexto comparativo con históricos o con el mercado, su valor en sí mismo es solo un dato más. Sin embargo, en relación con el PER, indica que la valoración del mercado está considerando un crecimiento futuro significativo.

- **FCF (nan), Net Income (nan):**  
  La ausencia de datos de flujo de caja libre y beneficios netos puede ser un problema para un

In [69]:
df_resultado = pd.DataFrame([{
    "fecha": row["Fecha evento"],
    "ticker": row["Ticker"],
    "analisis_value": response.choices[0].message.content
}])

df_resultado.to_csv("analisis_value_aapl.csv", sep=";", index=False)

### **EXTRA.** Downloading TS Data for autoamted Pipeline

In [11]:
import os
import pandas as pd
import requests
from datetime import datetime

ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = "https://www.alphavantage.co/query"
SYMBOL = "AAPL"
START_YEAR = 2022
TODAY = datetime.today().strftime("%Y-%m-%d")

# === FUNCIONES DE DESCARGA ===
def fetch_data(params, filename):
    params["apikey"] = ALPHA_VANTAGE_API_KEY
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    if "Error Message" in data or "Note" in data:
        print(f"⚠️ Error o límite alcanzado para {filename}")
        return

    if "Time Series (Daily)" in data:
        df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient='index')
    elif "Monthly Adjusted Time Series" in data:
        df = pd.DataFrame.from_dict(data["Monthly Adjusted Time Series"], orient='index')
    elif "Weekly Adjusted Time Series" in data:
        df = pd.DataFrame.from_dict(data["Weekly Adjusted Time Series"], orient='index')
    elif "Time Series (5min)" in data:
        df = pd.DataFrame.from_dict(data["Time Series (5min)"], orient='index')
    elif "Technical Analysis: SMA" in data:
        df = pd.DataFrame.from_dict(data["Technical Analysis: SMA"], orient='index')
    elif "Technical Analysis: RSI" in data:
        df = pd.DataFrame.from_dict(data["Technical Analysis: RSI"], orient='index')
    elif "Meta Data" in data:
        df = pd.DataFrame(data.items(), columns=["Field", "Value"])
    else:
        df = pd.DataFrame.from_dict(data, orient='index')
    
    df.to_csv(f"{SYMBOL}_{filename}.csv")
    print(f"✅ {filename} guardado")

# === LLAMADAS ===
fetch_data({"function": "TIME_SERIES_DAILY_ADJUSTED", "symbol": SYMBOL, "outputsize": "full"}, "daily_adjusted")
fetch_data({"function": "TIME_SERIES_WEEKLY_ADJUSTED", "symbol": SYMBOL}, "weekly_adjusted")
fetch_data({"function": "TIME_SERIES_MONTHLY_ADJUSTED", "symbol": SYMBOL}, "monthly_adjusted")
fetch_data({"function": "INCOME_STATEMENT", "symbol": SYMBOL}, "income_statement")
fetch_data({"function": "BALANCE_SHEET", "symbol": SYMBOL}, "balance_sheet")
fetch_data({"function": "CASH_FLOW", "symbol": SYMBOL}, "cash_flow")
fetch_data({"function": "OVERVIEW", "symbol": SYMBOL}, "overview")
fetch_data({"function": "EARNINGS", "symbol": SYMBOL}, "earnings")
fetch_data({"function": "SMA", "symbol": SYMBOL, "interval": "daily", "time_period": 200, "series_type": "close"}, "sma_200")
fetch_data({"function": "RSI", "symbol": SYMBOL, "interval": "daily", "time_period": 14, "series_type": "close"}, "rsi_14")


✅ daily_adjusted guardado
✅ weekly_adjusted guardado
✅ monthly_adjusted guardado
✅ income_statement guardado
✅ balance_sheet guardado
✅ cash_flow guardado
✅ overview guardado
✅ earnings guardado
✅ sma_200 guardado
✅ rsi_14 guardado


### Intrinsic VALUE - Benjamin Graham

#### 📌 1. ¿Qué es el crecimiento esperado (g) en la fórmula de Graham?
En el método de Graham, el crecimiento esperado **g** se refiere al **crecimiento anual estimado de los beneficios (EPS)** durante los próximos 5 años.

**💡 Técnicamente sí**, se interpreta como el **CAGR (Compound Annual Growth Rate)** del EPS a 5 años:

- **g = CAGR de EPS estimado a 5 años**
Es decir, cuánto se espera que crezca el beneficio anual de la empresa cada año durante los próximos 5 años.

**📍¿De dónde se saca?**
- De sitios financieros como:

    - Yahoo Finance → pestaña Analysis

    - Finbox, Morningstar, Simply Wall St

    - O informes de analistas

- En APIs: no siempre está disponible directamente, pero algunos ofrecen EarningsGrowth o Growth5Y.

**🧠 Si no está disponible, se suele asumir un valor estándar:**
- 5% si es conservador

- 10% como moderado

- 15–20% para empresas de crecimiento

#### 📌 2. ¿Qué es el Bond Yield (Y) en la fórmula de Graham?
En la fórmula clásica de Graham:

    Y representa el rendimiento actual del bono del tesoro AAA a 10 años (usado como tasa libre de riesgo)

🧠 En la fórmula original se usaba el valor de 4.4 como referencia del rendimiento promedio de los bonos en los años de Graham (1950s), por eso el ajuste con el valor actual de Y.

**📍 ¿De dónde se obtiene el rendimiento del bono AAA (Y)?**
**Fuentes confiables:**

- Moody's AAA Bond Yield

- US Treasury Yield Curve

- Google: “AAA bond yield today”

- FRED (Federal Reserve Economic Data)

**Ejemplo:**
En abril 2025, el AAA corporate bond yield ronda el 3.8%–4.0%, así que:

In [25]:
import pandas as pd
from datetime import datetime

# ✅ Diccionario con el crecimiento estimado a 5 años (CAGR EPS) para las 7 Magníficas
crecimientos_5y = {
    "AAPL": 12.0,
    "MSFT": 14.0,
    "GOOGL": 15.0,
    "AMZN": 25.0,
    "META": 18.0,
    "NVDA": 30.0,
    "TSLA": 27.0
}

BONO_YIELD = 3.8  # rendimiento actual de bonos AAA

# 📥 Funciones
def cargar_overview(path):
    df = pd.read_csv(path)
    return dict(zip(df.iloc[:, 0], df.iloc[:, 1]))

def obtener_precio_mas_reciente(path):
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date', ascending=False)
    fila = df.iloc[0]
    return float(fila.get('5. adjusted close', fila.get('4. close')))

def calcular_valor_intrinseco(eps, g, bono_yield):
    return round((eps * (8.5 + 2 * g) * 4.4) / bono_yield, 2)

# 🔁 Pedimos el ticker por input
ticker = input("Introduce el ticker de la empresa (ej. AAPL, MSFT, GOOGL): ").upper()

# ✅ Configuramos paths y valores
overview_path = f"C:/Users/Victor/Downloads/MSc DataScience/MasterThesis/data/{ticker}/fundamentals/{ticker}_overview.csv"
price_path = f"C:/Users/Victor/Downloads/MSc DataScience/MasterThesis/data/{ticker}/prices/{ticker}_daily_adjusted.csv"
CRECIMIENTO_ESTIMADO_5Y = crecimientos_5y.get(ticker, 10.0)  # valor por defecto si no está en la lista

# 📈 Proceso
try:
    overview = cargar_overview(overview_path)
    eps = float(overview.get("EPS", 0))
    per = float(overview.get("PERatio", 0))
    precio_actual = obtener_precio_mas_reciente(price_path)

    valor_intrinseco = calcular_valor_intrinseco(eps, CRECIMIENTO_ESTIMADO_5Y, BONO_YIELD)

    # 🧾 Resultados
    print(f"\n🧮 Ticker: {ticker}")
    print(f"🧮 EPS: {eps}")
    print(f"📈 PER: {per}")
    print(f"📉 Precio actual (ajustado): {precio_actual}")
    print(f"📐 Valor intrínseco (Graham): {valor_intrinseco}")
    print(f"📊 CAGR 5Y usado: {CRECIMIENTO_ESTIMADO_5Y}%")

    if valor_intrinseco > precio_actual:
        print("✅ La acción podría estar INFRAVALORADA.")
    else:
        print("❌ La acción podría estar SOBREVALORADA.")

except FileNotFoundError:
    print(f"❌ No se encontró alguno de los archivos para {ticker}. Verifica que exista la ruta correcta.")
except Exception as e:
    print(f"❌ Error procesando los datos de {ticker}: {e}")



🧮 Ticker: NVDA
🧮 EPS: 2.94
📈 PER: 36.2
📉 Precio actual (ajustado): 106.43
📐 Valor intrínseco (Graham): 233.19
📊 CAGR 5Y usado: 30.0%
✅ La acción podría estar INFRAVALORADA.
